# 머신 러닝의 네 가지 분류
1) 지도 학습

- 시퀀스 생성 : 사진이 주어지면 이를 설명하는 캡션을 생성합니다. 시퀀스 생성은 이따금 (시퀀스에 있는 단어나 토큰을 반복적으로 예측하는 것처럼) 일련의 분류 문제로 재구성할 수 있습니다.
- 구문 트리 예측 : 문장이 주어지면 분해된 구문 트리를 예측합니다.
- 물체 감지 : 사진이 주어지면 사진 안의 특정 물체 주위에 경계 상자를 그립니다. 이는 (많은 경계 상자 후보가 주어졌을 때 각 상자의 내용을 분류하는) 분류 문제로 표현되거나, 경계 상자의 좌표를 벡터 회귀로 예측하는 휘귀와 분류가 결합된 문제로 표현할 수 있습니다.
- 이미지 분할 : 사진이 주어졌을 때 픽셀 단위로 특정 물체에 마스킹을 합니다.

<br/>

2) 비지도 학습
- 차원 축소
- 군집

<br/>

3) 자기 지도 학습
사람이 만든 레이블을 사용하지 않는 지도 학습
- 오토 인코더

<br/>

4) 강화 학습
자율 주행 자동차, 자원 관리, 교육 등

# 머신 러닝 모델 평가

## 단순 홀드아웃 검증

In [1]:
num_validation_samples = 10000

np.random.shuffle(data)
validation_data = data[:num_validation_samples]
data = data[num_validation_samples:]

training_data = data[:]

model = get.model()
model.train(training_data)
validation_score = model.evaluate(validation_data)

# 여기에서 모델을 튜닝하고,
# 다시 훈련하고, 평가하고, 또 다시 튜닝하고...

model = get_model()
model.train(np.concatenate([training_data, validation_data]))
test_score = model.evaluate(test_data)

NameError: name 'data' is not defined

## K-겹 교차 검증

In [2]:
k = 4
num_validation_samples = len(data) // k

np.random.shuffle(data)

validation_scores = []
for fold in range(k):
    validation_data = data[num_validation_samples * fold: num_validation_samples * (fold + 1)]
    training_data = data[:num_validation_samples * fold] + data[num_validation_samples * (fold + 1):]
    
    model = get_model()
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)
    
    validation_score = np.average(validation_scores)
    
    model = get_model()
    model.train(data)
    test_score = model.evaluate(test_data)

NameError: name 'data' is not defined

## 셔플링을 사용한 반복 K-겹 교차 검증
데이터가 적고 가능한 정확하게 모델을 평가하고자 할 때 사용합니다. 캐글 경연에서는 이 방법이 아주 크게 도움이 됩니다. 이 방법은 K-겹 교차 검증을 여러 번 적용하되 K개의 분할로 나누기 전에 매번 데이터를 무작위로 섞습니다.

## 기억해야 할 것
- 대표성 있는 데이터 : 훈련 세트와 테스트 세트가 주어진 데이터에 대한 대표성이 있어야 합니다. 훈련 세트와 테스트 세트로 나누기 전에 데이터를 무작위로 섞는 것이 일반적입니다.
- 시간의 방향 : 과거로부터 미래를 예측하려고 한다면(예를 들어 내일의 날씨, 주식 시세 등) 데이터를 분할하기 전에 무작위로 섞어서는 절대 안 됩니다. 이렇게 하면 미래의 정보가 누설되기 때문입니다. 즉 모델이 사실상 미래 데이터에서 훈련될 것입니다. 이런 문제에서는 훈련 세트에 있는 데이터보다 테스트 세트에 있는 모든 데이터가 미래의 것이어야 합니다.
- 데이터 중복 : 한 데이터셋에 어떤 데이터 포인트가 두 번 등장하면(실제 데이터셋에서 아주 흔한 일입니다.), 데이터를 섞고 훈련 세트와 검증 세트로 나누었을 때 훈련 세트와 검증 세트에 데이터 포인트가 중복될 수 있습니다. 이로 인해 훈련 데이터의 일부로 테스트하는 최악의 경우가 됩니다. 훈련 세트와 검증 세트가 중복되지 않는지 확인하세요.

# 데이터 전처리, 특성 공학, 특성 학습

## 신경망을 위한 데이터 전처리

- 벡터화 : 신경망에서 모든 입력과 타깃은 부동 소수 데이터로 이루어진 텐서여야 합니다. 사운드, 이미지, 텍스트 등 처리해야 할 것이 무엇이든지 먼저 텐서로 변환해야 합니다. 이 단계를 데이터 벡터화라고 합니다.

- 값 정규화 : 이미지 데이터를 다룰 때 최종적으로 0~1 사이의 부동 소수 값으로 만들었습니다. 데이터를 네트워크에 주입하기 전에 각 특성을 독립적으로 정규화하여 평균이 0이고 표준편차가 1이 되도록 만들었습니다.

- 누락된 값 다루기 : 일반적으로 신경망에서 0이 사전에 정의된 의미 있는 값이 아니라면 누락된 값을 0으로 입력해도 괜찮습니다.

## 특성 공학
특성 공학은 머신 러닝 알고리즘에 관한 지식을 사용하는 단계입니다. 모델에 데이터를 주입하기 전에 하드코딩된 변환을 적용하여 알고리즘이 더 잘 수행되도록 만들어 줍니다. 특성을 더 간단한 방식으로 표현하여 문제를 쉽게 만드는 것이 특성 공학의 핵심이다.

# 과대적합과 과소적합
머신 러닝의 근본적인 이슈는 최적화와 일반화 사이의 줄다리기입니다. 최적화는 가능한 훈련 데이터에서 최고의 성능을 얻으려고 모델을 조정하는 과정입니다. 반면에 일반화는 훈련된 모델이 이전에 본 적 없는 데이터에서 얼마나 잘 수행되는지 의미합니다. 모델이 관련성이 없고 좋지 못한 패턴을 훈련 데이터에서 학습하지 못하도록 하려면 가장 좋은 방법은 더 많은 훈련 데이터를 모으는 것입니다.

## 네트워크 크기 축소
과대적합을 막는 가장 단순한 방법은 모델의 크기, 즉 모델에 있는 학습 파라미터의 수를 줄이는 것입니다. 파라미터가 많은 모델이 기억 용량이 더 많습니다. 훈련 샘플과 타깃 사이를 딕셔너리 같은 일대일 매핑으로 완벽하게 학습할 수도 있습니다. 적절한 모델 크기를 찾는 일반적인 작업 흐름은 비교적 적은 수의 층과 파라미터로 시작합니다. 그다음 검증 손실이 감소되기 시작할 때까지 층이나 유닛의 수를 늘리는 것입니다.

In [6]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [4]:
# 더 작은 네트워크로 바꾸어 보죠
model = models.Sequential()
model.add(layers.Dense(6, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(6, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

## 가중치 규제 추가
오캄의 면도날 이론 : 어떤 것에 대한 두 가지의 설명이 있다면 더 적은 가정이 필요한 간단한 설명이 옳을 것이라는 이론
과대적합을 완화하기 위한 일반적인 방법은 네트워크의 복잡도에 제한을 두어 가중치가 작은 값을 가지도록 강제하는 것입니다. 가중치 값의 분포가 더 균일하게 됩니다. 이를 가중치 규제라고 하며, 네트워크의 손실 함수에 큰 가중치에 연관된 비용을 추가합니다.

- L1 규제 : 가중치의 절댓값에 비례하는 비용이 추가됩니다.(MAE)
- L2 규제 : 가중치의 제곱에 비례하는 비용이 추가됩니다.(MSE) L2 규제는 신경망에서 가중치 감쇠라고도 부릅니다. 

In [7]:
from keras import regularizers

model = models.Sequential()
model.add(layers.Dense(16, kernel_initializer=regularizers.l2(0.001), activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, kernel_initializer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

TypeError: Expected int32, got 0.0 of type 'float' instead.

## 드롭아웃 추가
네트워크 층에 드롭아웃을 적용하면 훈련하는 동안 무작위로 층의 일부 출력 특성을 제외시킵니다.(0으로 만듭니다) 드롭아웃 비율은 0이 될 특성의 비율입니다. 보통 0.2에서 0.5 사이로 지정됩니다.

In [9]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

## 정리
- 훈련 데이터를 더 모읍니다.
- 네트워크의 용량을 감소시킵니다.
- 가중치 규제를 추가합니다.
- 드롭아웃을 추가합니다.

# 보편적인 머신 러닝 작업 흐름

## 문제 정의와 데이터셋 수집

- 입력 데이터는 무엇인가요? 어떤 것을 예측하려고 하나요? 가용한 훈련 데이터가 있어야 어떤 것을 예측하도록 학습할 수 있습니다. 예를 들어 영화 리뷰와 감성 레이블이 태깅되어 있어야 영화 리뷰의 감성 분류를 학습할 수 있습니다. 이런 식으로 보통 가용 데이터의 유무는 이 단계에서 제한 요소가 됩니다(데이터를 수집할 비용이 없다면).
- 당면한 문제가 어떤 종류인가요? 이진 분류인가요? 다중 분류인가요? 스칼라 회귀인가요? 벡터 회귀인가요? 다중 레이블 다중 분류인가요? 아니면 군집, 생성 또는 강화 학습 같은 다른 문제인가요? 문제의 유형을 식별하면 모델의 구조와 손실 함수 등을 선택하는 데 도움이 됩니다.

- 주어진 입력으로 예측할 수 있다고 가설을 세웁니다.
- 가용한 데이터에 입력과 출력 사이의 관계를 학습하는 데 충분한 정보가 있다고 가설을 세웁니다.

## 성공 지표 선택
클래스 분포가 균일한 분류 문제에서는 정확도와 ROC AUC가 일반적인 지표입니다. 클래스 분포가 균일하지 않은 문제에서는 정밀도와 재현율을 사용할 수 있습니다. 랭킹 문제나 다중 레이블 문제에는 평균 정밀도를 사용할 수 있습니다.

## 평가 방법 선택
- 홀드아웃 검증 세트 분리 : 데이터가 풍부할 때 사용합니다.
- K-겹 교차 검증 : 홀드아웃 검증을 사용하기에 샘플의 수가 너무 적을 때 사용합니다.
- 반복 K-겹 교차 검증 : 데이터가 적고 매우 정확한 모델 평가가 필요할 때 사용합니다.

## 데이터 준비
- 앞서 보았듯이 데이터는 텐서로 구성됩니다.
- 이 텐서에 있는 값은 일반적으로 작은 값으로 스케일 조정되어 있습니다. 예를 들어 [-1, 1]이나 [0, 1] 범위입니다.
- 특성마다 범위가 다르면(여러 종류의 값으로 이루어진 데이터라면) 정규화해야 합니다.
- 특성 공학을 수행할 수 있습니다. 특히 데이터가 적을 때입니다.

## 기본보다 나은 모델 훈련하기
이 단계의 목표는 통계적 검정력을 달성하는 것입니다. 즉 아주 단순한 모델보다 나은 수준의 작은 모델을 개발합니다.
- 마지막 층의 활성화 함수 : 네트워크의 출력에 필요한 제한을 가합니다.
- 손실 함수 : 풀려고 하는 문제의 종류에 적합해야 합니다.
- 최적화 설정 : 대부분의 경우 rmsprop과 기본 학습률을 사용하는 것이 무난합니다.

| 문제 유형                 | 마지막 층의 활성화 함수 | 손실 함수                    |
|---------------------------|-------------------------|------------------------------|
| 이진 분류                 | 시그모이드              | binary_crossentropy          |
| 단일 레이블 다중 분류     | 소프트맥스              | categorical_crossentropy     |
| 다중 레이블 다중 분류     | 시그모이드              | binary_crossentropy          |
| 임의 값에 대한 회귀       | 없음                    | mse                          |
| 0과 1 사이 값에 대한 회귀 | 시그모이드              | mse 또는 binary_crossentropy |

## 몸집 키우기: 과대적합 모델 구축
1. 층을 추가합니다.
2. 층의 크기를 키웁니다.
3. 더 많은 에포크 동안 훈련합니다.

## 모델 규제와 하이퍼파라미터 튜닝

- 드롭아웃을 추가합니다.
- 층을 추가하거나 제거해서 다른 구조를 시도해 봅니다.
- L1이나 L2 또는 두 가지 모두 추가합니다.
- 최적의 설정을 찾기 위해 하이퍼파라미터를 바꾸어 시도해 봅니다(층의 유닛 수나 옵티마이저의 학습률 등)
- 선택적으로 특성 공학을 시도해 봅니다. 새로운 특성을 추가하거나 유용하지 않을 것 같은 특성을 제거합니다.

# 요약

- 주어진 문제와 훈련할 데이터를 정의합니다. 이 데이터를 수집하고 필요하면 레이블을 태깅합니다.
- 성공을 어떻게 측정할지 선택하세요. 검증 데이터에서 모니터링할 지표는 무엇인가요?
- 평가 방법을 결정하세요. 홀드아웃 검증이나 K-겹 교차 검증인가요? 검증에 사용해야 할 데이터의 양은 얼마나 되나요?
- 단순한 랜덤 선택 모델보다 나은 통계적 검정력이 있는 첫 번째 모델을 만듭니다.
- 과대적합된 모델을 만듭니다.
- 검증 데이터의 성능에 기초하여 모델에 규제를 적용하고 하이퍼파라미터를 튜닝합니다. 머신 러닝 연구의 대부분은 이 단계에 집중됩니다. 하지만 큰 그림을 놓치지 마세요.